To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**NEW** Unsloth now supports training the new **gpt-oss** model from OpenAI! You can start finetune gpt-oss for free with our **[Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)**!

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_tesla_t4 = False
    get_vllm, get_triton = ("vllm==0.10.1", "triton==3.2.0") if is_t4 else ("vllm", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers transformers
    !uv pip install -qqq {get_triton}

### Unsloth

Load up `Llama 3.1 8B Instruct`, and set parameters

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",  # ← CHANGED: Smaller model
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-26 10:57:50 [__init__.py:241] Automatically detected platform cuda.
ERROR 08-26 10:57:51 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.4. vLLM: 0.10.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-3b-instruct-bnb-4bit with actual GPU utilization = 59.

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.8.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [ ]:
import re
from datasets import load_dataset, Dataset
import random

In [ ]:
MATH_SYSTEM_PROMPT = """
Solve this math problem step by step.
<reasoning>
Show your work and thinking process here.
</reasoning>
<answer>
Provide the final numerical answer here.
</answer>
"""

DIALOGUE_SYSTEM_PROMPT = """
Respond helpfully and conversationally to the user's message.
"""

In [ ]:
def extract_xml_answer(text: str) -> str:
    """Extract answer from XML format"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    """Extract answer from GSM8K format (#### answer)"""
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def extract_number_from_text(text: str) -> float:
    """Extract first number from text, return 0 if none found"""
    import re
    numbers = re.findall(r'-?\d+\.?\d*', text.replace(',', ''))
    if numbers:
        try:
            return float(numbers[0])
        except:
            return 0.0
    return 0.0

In [ ]:
def get_gsm8k_dataset(split="train", max_samples=1000):
    """Load GSM8K math dataset"""
    data = load_dataset('openai/gsm8k', 'main')[split]

    # Limit dataset size for quick experiments
    if max_samples:
        indices = random.sample(range(len(data)), min(max_samples, len(data)))
        data = data.select(indices)

    def format_math_sample(x):
        return {
            'task_type': 'math',
            'prompt': [
                {'role': 'system', 'content': MATH_SYSTEM_PROMPT},
                {'role': 'user', 'content': x['question']}
            ],
            'ground_truth': extract_hash_answer(x['answer']),
            'numerical_answer': extract_number_from_text(extract_hash_answer(x['answer']) or "0")
        }

    return data.map(format_math_sample)

In [ ]:
def get_dialogue_dataset(split="train", max_samples=1000):
    """Load Alpaca dialogue dataset"""
    data = load_dataset('tatsu-lab/alpaca')[split]

    # Limit dataset size for quick experiments
    if max_samples:
        indices = random.sample(range(len(data)), min(max_samples, len(data)))
        data = data.select(indices)

    def format_dialogue_sample(x):
        # Combine instruction and input if input exists
        user_message = x['instruction']
        if x.get('input', '').strip():
            user_message = f"{x['instruction']}\n\nInput: {x['input']}"

        return {
            'task_type': 'dialogue',
            'prompt': [
                {'role': 'system', 'content': DIALOGUE_SYSTEM_PROMPT},
                {'role': 'user', 'content': user_message}
            ],
            'ground_truth': x['output'],
            'numerical_answer': None  # No numerical answer for dialogue
        }

    return data.map(format_dialogue_sample)

In [ ]:
print("Loading datasets...")
math_dataset = get_gsm8k_dataset(max_samples=800)
dialogue_dataset = get_dialogue_dataset(max_samples=800)

print(f"Math dataset: {len(math_dataset)} samples")
print(f"Dialogue dataset: {len(dialogue_dataset)} samples")

# Sample check
print("\n" + "="*50)
print("MATH SAMPLE:")
print("Question:", math_dataset[0]['prompt'][1]['content'][:500] + "...")
print("Answer:", math_dataset[0]['ground_truth'])

print("\n" + "="*50)
print("DIALOGUE SAMPLE:")
print("Instruction:", dialogue_dataset[0]['prompt'][1]['content'][:500] + "...")
print("Expected:", dialogue_dataset[0]['ground_truth'][:1000] + "...")

Loading datasets...


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

...

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Math dataset: 800 samples
Dialogue dataset: 800 samples

MATH SAMPLE:
Question: A man is returning home from work and trying to decide which route to take.  His first route option includes 3 stoplights.  This route will take him 10 minutes if all three lights are green, but each light that is red will add 3 minutes to the trip.  The second route does not include any stoplights and takes 14 minutes.  If the man chooses the first route, how much longer will his trip be if all 3 stoplights are red?...
Answer: 5

DIALOGUE SAMPLE:
Instruction: Create an algorithm that can sort an array of numbers into ascending order.

Input: [5, 2, 8, 1, 4]...
Expected: """
Algorithm to sort an array of numbers into ascending order
"""

def sort_ascending(nums):
    sorted_list = []
    while nums:
        lowest = nums[0]
        for num in nums:
            if num < lowest:
                lowest = num
        sorted_list.append(lowest)
        nums.remove(lowest)

    return sorted_list

if __name__ == 

In [ ]:
import google.generativeai as genai
import os
from typing import List
import time

GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
genai.configure(api_key=GEMINI_API_KEY)

# Initialize Gemini model
try:
    gemini_model = genai.GenerativeModel('gemini-1.5-flash')  # Fast and cheap model
    print("Gemini API initialized successfully!")
except Exception as e:
    print(f"Gemini API setup failed: {e}")
    print("Please add your API key above!")

Gemini API initialized successfully!


In [ ]:
def evaluate_dialogue_with_gemini(question: str, response: str) -> float:
    """Use Gemini to evaluate dialogue response quality (0-10 scale)"""

    evaluation_prompt = f"""
You are evaluating an AI assistant's response to a user question. Rate the response on a scale of 0-10 based on:
- Helpfulness: Does it answer the question?
- Accuracy: Is the information correct?
- Clarity: Is it easy to understand?
- Completeness: Does it fully address the question?
- Tone: Is it appropriate and professional?

Question: {question}

Response: {response}

Provide ONLY a number between 0-10 (can use decimals like 7.5). No explanation needed.
"""

    try:
        # Call Gemini API
        result = gemini_model.generate_content(evaluation_prompt)

        # Extract numerical score
        score_text = result.text.strip()
        score = float(score_text)

        # Ensure score is in valid range
        score = max(0.0, min(10.0, score))

        return score

    except Exception as e:
        print(f"Gemini evaluation error: {e}")
        return 5.0  # Fallback to neutral score

def batch_evaluate_dialogues(prompts: List, responses: List[str], batch_delay=1.0) -> List[float]:
    """Evaluate multiple responses with rate limiting"""
    scores = []

    for i, (prompt, response) in enumerate(zip(prompts, responses)):
        question = prompt[-1]['content']  # Get user question
        score = evaluate_dialogue_with_gemini(question, response)
        scores.append(score)

        # Rate limiting - don't spam the API
        if i < len(responses) - 1:  # Don't delay after last request
            time.sleep(batch_delay)

        # Progress indicator
        if (i + 1) % 5 == 0:
            print(f"Evaluated {i + 1}/{len(responses)} responses...")

    return scores

In [ ]:
def math_hard_reward(prompts, completions, answer, numerical_answer, **kwargs) -> list[float]:
    """Hard reward for math: Exact match = 1.0, wrong = 0.0"""
    responses = [completion[0]['content'] for completion in completions]
    extracted_answers = [extract_xml_answer(r) for r in responses]

    rewards = []
    for i, extracted in enumerate(extracted_answers):
        # Try exact string match first
        if extracted == answer[i]:
            rewards.append(2.0)
        # Try numerical match as fallback
        elif numerical_answer and numerical_answer[i] is not None:
            extracted_num = extract_number_from_text(extracted)
            if abs(extracted_num - numerical_answer[i]) < 0.01:  # Allow small rounding errors
                rewards.append(2.0)
            else:
                rewards.append(0.0)
        else:
            rewards.append(0.0)

    # Debug print for first completion
    if len(responses) > 0:
        print(f"\n[MATH HARD] Question: {prompts[0][-1]['content'][:60]}...")
        print(f"Expected: {answer[0]} | Got: {extracted_answers[0]} | Reward: {rewards[0]}")

    return rewards

In [ ]:
def dialogue_hard_reward_gemini(prompts, completions, **kwargs) -> list[float]:
    """Hard reward using Gemini: Good (≥7) = 2.0, Poor (<7) = 0.0"""
    responses = [completion[0]['content'] for completion in completions]

    # Get Gemini scores (0-10)
    gemini_scores = batch_evaluate_dialogues(prompts, responses)

    # Convert to hard rewards (binary)
    hard_rewards = [2.0 if score >= 7.0 else 0.0 for score in gemini_scores]

    # Debug print
    if len(responses) > 0:
        print(f"\n[DIALOGUE HARD GEMINI] Score: {gemini_scores[0]:.1f}/10 | Reward: {hard_rewards[0]}")
        print(f"Response: {responses[0][:1000]}...")

    return hard_rewards

In [ ]:
def math_perplexity_reward(prompts, completions, **kwargs) -> list[float]:
   """Perplexity-based reward: Lower perplexity = higher reward"""

   # Get the model and tokenizer from kwargs (passed by GRPO trainer)
   model = kwargs.get('model')
   tokenizer = kwargs.get('tokenizer')

   if model is None or tokenizer is None:
       print("Warning: Model/tokenizer not available for perplexity calculation")
       return [1.0] * len(completions)  # Fallback to neutral rewards

   rewards = []

   for i, completion in enumerate(completions):
       try:
           # Get the prompt text
           prompt_text = tokenizer.apply_chat_template(prompts[i], tokenize=False, add_generation_prompt=False)

           # Get completion text
           response_text = completion[0]['content']

           # Calculate perplexity for the completion given the prompt
           full_sequence = prompt_text + response_text

           # Tokenize and calculate loss
           inputs = tokenizer(full_sequence, return_tensors="pt", truncation=True, max_length=512)
           inputs = {k: v.to(model.device) for k, v in inputs.items()}

           with torch.no_grad():
               outputs = model(**inputs, labels=inputs['input_ids'])
               loss = outputs.loss
               perplexity = torch.exp(loss)

           # Convert perplexity to reward (lower perplexity = higher reward)
           # Scale factor of 50 keeps rewards in reasonable range
           reward = 2.0 / (1.0 + perplexity.item() / 50.0)

           rewards.append(reward)

       except Exception as e:
           print(f"Perplexity calculation error: {e}")
           rewards.append(1.0)  # Neutral fallback

   # Debug print
   if len(rewards) > 0:
       print(f"\n[MATH PERPLEXITY] Perplexity reward: {rewards[0]:.3f}")

   return rewards

In [ ]:
def dialogue_perplexity_reward(prompts, completions, **kwargs) -> list[float]:
   """Perplexity-based reward for dialogue: Lower perplexity = higher reward"""

   # Get the model and tokenizer from kwargs (passed by GRPO trainer)
   model = kwargs.get('model')
   tokenizer = kwargs.get('tokenizer')

   if model is None or tokenizer is None:
       print("Warning: Model/tokenizer not available for perplexity calculation")
       return [1.0] * len(completions)  # Fallback to neutral rewards

   rewards = []

   for i, completion in enumerate(completions):
       try:
           # Get the prompt text
           prompt_text = tokenizer.apply_chat_template(prompts[i], tokenize=False, add_generation_prompt=False)

           # Get completion text
           response_text = completion[0]['content']

           # Calculate perplexity for the completion given the prompt
           full_sequence = prompt_text + response_text

           # Tokenize and calculate loss
           inputs = tokenizer(full_sequence, return_tensors="pt", truncation=True, max_length=512)
           inputs = {k: v.to(model.device) for k, v in inputs.items()}

           with torch.no_grad():
               outputs = model(**inputs, labels=inputs['input_ids'])
               loss = outputs.loss
               perplexity = torch.exp(loss)

           # Convert perplexity to reward (lower perplexity = higher reward)
           # Scale factor of 50 keeps rewards in reasonable range
           reward = 2.0 / (1.0 + perplexity.item() / 50.0)

           rewards.append(reward)

       except Exception as e:
           print(f"Perplexity calculation error: {e}")
           rewards.append(1.0)  # Neutral fallback

   # Debug print
   if len(rewards) > 0:
       print(f"\n[DIALOGUE PERPLEXITY] Perplexity reward: {rewards[0]:.3f}")

   return rewards

In [ ]:
def format_reward_func(completions, **kwargs) -> list[float]:
    """Reward proper XML formatting"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []

    for response in responses:
        score = 0.0
        if '<reasoning>' in response and '</reasoning>' in response:
            score += 0.25
        if '<answer>' in response and '</answer>' in response:
            score += 0.25
        rewards.append(score)

    return rewards

In [ ]:
def test_gemini_evaluation():
    """Test if Gemini evaluation works"""
    test_question = "What is the capital of France?"
    test_response = "The capital of France is Paris."

    print("Testing Gemini evaluation...")
    score = evaluate_dialogue_with_gemini(test_question, test_response)
    print(f"Test Score: {score}/10")

    if score > 8.0:
        print("Gemini evaluation working well!")
    else:
        print("Gemini evaluation might need adjustment")

In [ ]:
test_gemini_evaluation()

Testing Gemini evaluation...
Test Score: 10.0/10
Gemini evaluation working well!


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer
import os

max_prompt_length = 256

In [ ]:
base_config = {
    "learning_rate": 5e-6,
    "adam_beta1": 0.9,
    "adam_beta2": 0.99,
    "weight_decay": 0.1,
    "warmup_ratio": 0.1,
    "lr_scheduler_type": "cosine",
    "optim": "paged_adamw_8bit",
    "logging_steps": 1,
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 2,  # Increased for stability
    "num_generations": 4,  # Reduced to save memory
    "max_prompt_length": max_prompt_length,
    "max_completion_length": max_seq_length - max_prompt_length,
    "max_steps": 100,  # Reduced for quick comparison
    "save_steps": 50,
    "max_grad_norm": 0.1,
    "report_to": "none",
}

In [ ]:
math_hard_config = GRPOConfig(
    **base_config,
    output_dir = "outputs/math_hard_rewards",
    run_name = "math_hard_rewards"
)

math_perplexity_config = GRPOConfig(
    **base_config,
    output_dir = "outputs/math_perplexity_rewards",
    run_name = "math_perplexity_rewards"
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4
Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
dialogue_hard_config = GRPOConfig(
    **base_config,
    output_dir = "outputs/dialogue_hard_rewards",
    run_name = "dialogue_hard_rewards"
)

dialogue_perplexity_config = GRPOConfig(
    **base_config,
    output_dir = "outputs/dialogue_perplexity_rewards",
    run_name = "dialogue_perplexity_rewards"
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4
Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
# Math reward mappings
MATH_HARD_REWARDS = [math_hard_reward, format_reward_func]
MATH_PERPLEXITY_REWARDS = [math_perplexity_reward, format_reward_func]

# Dialogue reward mappings
DIALOGUE_HARD_REWARDS = [dialogue_hard_reward_gemini, format_reward_func]
DIALOGUE_PERPLEXITY_REWARDS = [dialogue_perplexity_reward, format_reward_func]

In [ ]:
training_experiments = [
    {
        "name": "Math Hard Rewards",
        "dataset": math_dataset,
        "config": math_hard_config,
        "rewards": MATH_HARD_REWARDS,
        "task_type": "math"
    },
    {
        "name": "Math Perplexity Rewards",
        "dataset": math_dataset,
        "config": math_perplexity_config,
        "rewards": MATH_PERPLEXITY_REWARDS,
        "task_type": "math"
    },
    {
        "name": "Dialogue Hard Rewards",
        "dataset": dialogue_dataset,
        "config": dialogue_hard_config,
        "rewards": DIALOGUE_HARD_REWARDS,
        "task_type": "dialogue"
    },
    {
        "name": "Dialogue Perplexity Rewards",
        "dataset": dialogue_dataset,
        "config": dialogue_perplexity_config,
        "rewards": DIALOGUE_PERPLEXITY_REWARDS,
        "task_type": "dialogue"
    }
]

In [ ]:
print("Training Configurations Ready!")
print(f"{len(training_experiments)} experiments planned:")
for exp in training_experiments:
    print(f"   • {exp['name']}: {len(exp['dataset'])} samples")

print(f"\nEach experiment: {base_config['max_steps']} steps")

Training Configurations Ready!
4 experiments planned:
   • Math Hard Rewards: 800 samples
   • Math Perplexity Rewards: 800 samples
   • Dialogue Hard Rewards: 800 samples
   • Dialogue Perplexity Rewards: 800 samples

Each experiment: 100 steps


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
from trl import GRPOTrainer
import torch
import time
import json

In [ ]:
# Create wrapped reward functions that have access to model/tokenizer
def create_model_aware_rewards(reward_funcs, model, tokenizer):
    wrapped_funcs = []
    for func in reward_funcs:
        def wrapped_func(*args, **kwargs):
            kwargs['model'] = model
            kwargs['tokenizer'] = tokenizer
            return func(*args, **kwargs)
        wrapped_funcs.append(wrapped_func)
    return wrapped_funcs

In [ ]:
training_results = {}

def run_single_experiment(experiment_config):
    """Run a single training experiment"""
    name = experiment_config["name"]
    dataset = experiment_config["dataset"]
    config = experiment_config["config"]
    rewards = experiment_config["rewards"]
    task_type = experiment_config["task_type"]

    print(f"\nStarting: {name}")
    print(f"Dataset: {len(dataset)} samples")
    print(f"Task: {task_type}")
    print(f"Rewards: {len(rewards)} functions")
    print("-" * 50)

    # Create trainer
    # Use wrapped functions in trainer
    model_aware_rewards = create_model_aware_rewards(rewards, model, tokenizer)
    trainer = GRPOTrainer(
        model=model,
        processing_class=tokenizer,
        reward_funcs=model_aware_rewards,  # Now has model access
        args=config,
        train_dataset=dataset,
    )

    # Train and track results
    start_time = time.time()
    trainer.train()
    end_time = time.time()

    # Store results
    training_results[name] = {
        "task_type": task_type,
        "training_time": end_time - start_time,
        "final_step": config.max_steps,
        "output_dir": config.output_dir
    }

    print(f"Completed: {name}")
    print(f"Training time: {end_time - start_time:.1f} seconds")
    print(f"Results saved to: {config.output_dir}")

    return trainer

In [ ]:
trained_models = {}

# Option 1: Run all experiments (will take ~2-3 hours)
run_all = False  # Set to True when ready for full run

if run_all:
    for experiment in training_experiments:
        trained_models[experiment["name"]] = run_single_experiment(experiment)

        # Brief pause between experiments
        time.sleep(10)

    print("\nALL EXPERIMENTS COMPLETED!")

else:
    # Option 2: Run just one experiment first (for testing)
    print("TESTING MODE: Running single experiment")
    test_experiment = training_experiments[1]  # Math Hard Rewards

    print(f"Testing with: {test_experiment['name']}")
    trained_models["test"] = run_single_experiment(test_experiment)


TESTING MODE: Running single experiment
Testing with: Math Perplexity Rewards

Starting: Math Perplexity Rewards
Dataset: 800 samples
Task: math
Rewards: 2 functions
--------------------------------------------------


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 800 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,entropy,rewards / wrapped_func / mean,rewards / wrapped_func / std,rewards / wrapped_func / mean,rewards / wrapped_func / std
1,-0.000000,0.375000,0.144338,210.250000,117.000000,298.000000,0.000000,210.250000,117.000000,298.000000,0.000000,0,0.187500,0.115728,0.187500,0.115728
2,-0.000000,0.375000,0.144338,190.500000,142.000000,294.000000,0.000000,190.500000,142.000000,294.000000,0.000000,No Log,0.187500,0.115728,0.187500,0.115728
3,-0.000000,0.062500,0.125000,246.750000,181.000000,438.000000,0.000000,246.750000,181.000000,438.000000,0.000000,No Log,0.031250,0.088388,0.031250,0.088388
4,0.000000,0.250000,0.288675,223.750000,173.000000,296.000000,0.000000,223.750000,173.000000,296.000000,0.000009,No Log,0.125000,0.133631,0.125000,0.133631
5,0.000000,0.375000,0.250000,192.125000,131.000000,266.000000,0.000000,192.125000,131.000000,266.000000,0.000005,No Log,0.187500,0.115728,0.187500,0.115728
6,0.000000,0.187500,0.269338,328.375000,171.000000,768.000000,0.125000,265.571442,171.000000,359.000000,0.000009,No Log,0.093750,0.129387,0.093750,0.129387
7,0.000000,0.187500,0.269338,199.000000,155.000000,255.000000,0.000000,199.000000,155.000000,255.000000,0.000008,No Log,0.093750,0.129387,0.093750,0.129387
8,0.000000,0.375000,0.144338,188.125000,125.000000,331.000000,0.000000,188.125000,125.000000,331.000000,0.000006,No Log,0.187500,0.115728,0.187500,0.115728
9,0.000000,0.250000,0.288675,205.250000,114.000000,312.000000,0.000000,205.250000,114.000000,312.000000,0.000009,No Log,0.125000,0.133631,0.125000,0.133631
10,-0.000000,0.250000,0.250000,175.500000,127.000000,210.000000,0.000000,175.500000,127.000000,210.000000,0.000010,No Log,0.125000,0.133631,0.125000,0.133631


Completed: Math Perplexity Rewards
Training time: 3563.9 seconds
Results saved to: outputs/math_perplexity_rewards


In [ ]:
print("\nTRAINING RESULTS SUMMARY:")
print("-" * 40)
for name, results in training_results.items():
    print(f"{name}:")
    print(f"  • Task: {results['task_type']}")
    print(f"  • Time: {results['training_time']:.1f}s")
    print(f"  • Steps: {results['final_step']}")
    print(f"  • Output: {results['output_dir']}")
    print()


TRAINING RESULTS SUMMARY:
----------------------------------------
Math Perplexity Rewards:
  • Task: math
  • Time: 3563.9s
  • Steps: 100
  • Output: outputs/math_perplexity_rewards



In [ ]:
# Create zip file
import shutil
shutil.make_archive('outputs', 'zip', 'outputs')

# Download the zip
from google.colab import files
files.download('outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load the Math Soft trained model
from unsloth import FastLanguageModel

model_soft, tokenizer_soft = FastLanguageModel.from_pretrained(
    "outputs/math_soft_rewards/checkpoint-100"
)

# Test with the same problems you used for Math Hard
test_problems = [
    "Sarah has 24 stickers. She gives 8 to her friend and buys 15 more. How many stickers does she have now?",
    "A pizza is cut into 8 slices. If Tom eats 3 slices and Jerry eats 2 slices, how many slices are left?",
    "There are 45 students in a class. 18 are boys. How many girls are there?"
]

def test_soft_model(problems, model, tokenizer):
    print("TESTING MATH SOFT REWARDS MODEL:")
    print("-" * 50)

    for i, problem in enumerate(problems, 1):
        print(f"\nTest Problem {i}:")
        print(f"Question: {problem}")

        # Format prompt
        messages = [
            {"role": "system", "content": MATH_SYSTEM_PROMPT},
            {"role": "user", "content": problem}
        ]

        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        print(f"Response: {response}")

        # Check format and answer
        if "<reasoning>" in response and "<answer>" in response:
            print("Format: XML (learned)")
            answer = extract_xml_answer(response)
            print(f"Extracted Answer: {answer}")
        else:
            print("Format: Natural language")

        print("-" * 50)

# Run the test
test_soft_model(test_problems, model_soft, tokenizer_soft)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.4. vLLM: 0.10.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
TESTING MATH SOFT REWARDS MODEL:
--------------------------------------------------

Test Problem 1:
Question: Sarah has 24 stickers. She gives 8 to her friend and buys 15 more. How many stickers does she have now?
Response: <reasoning>
To find out how many stickers Sarah has now, we need to subtract the number of stickers she gave away from the initial number of stickers she had, then add the number of stickers she bought.

1. Initial number of stickers: 24
2. Number of stickers given away: 8
3. Number of stickers bought: 15

Fir

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
# Create zip file
import shutil
shutil.make_archive('grpo_saved_lora', 'zip', 'grpo_saved_lora')

# Download the zip
from google.colab import files
files.download('grpo_saved_lora.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>